# Laporan Analisis Bab 11: Training Deep Neural Networks

## Pendahuluan
[cite_start]Bab 11 membahas tantangan-tantangan yang muncul saat melatih Jaringan Saraf Tiruan (DNN) yang sangat dalam (*deep*)[cite: 331]. [cite_start]Bab sebelumnya telah memperkenalkan ANN dangkal dengan beberapa *hidden layer*, namun untuk masalah yang lebih kompleks (misalnya, deteksi objek pada gambar resolusi tinggi), DNN yang jauh lebih dalam diperlukan[cite: 331]. [cite_start]Pelatihan DNN yang dalam menghadapi berbagai kesulitan, termasuk masalah *vanishing/exploding gradients*, kurangnya data pelatihan berlabel, waktu pelatihan yang lambat, dan risiko *overfitting* yang tinggi karena jutaan parameter[cite: 331]. [cite_start]Bab ini menyajikan berbagai teknik dan strategi untuk mengatasi masalah-masalah tersebut[cite: 331].

## Ringkasan Isi Bab

Bab 11 menguraikan beberapa konsep kunci terkait Pelatihan DNN yang Dalam:

1.  **The Vanishing/Exploding Gradients Problems (Masalah Vanishing/Exploding Gradients)**:
    * [cite_start]**Vanishing Gradients**: Gradien menjadi semakin kecil saat algoritma bergerak mundur ke lapisan yang lebih rendah selama *backpropagation*[cite: 332]. [cite_start]Ini menyebabkan bobot koneksi di lapisan bawah hampir tidak berubah, sehingga pelatihan gagal konvergen ke solusi yang baik[cite: 332].
    * [cite_start]**Exploding Gradients**: Kebalikannya, gradien bisa menjadi sangat besar, menyebabkan pembaruan bobot yang sangat besar dan algoritma menjadi divergen[cite: 332]. [cite_start]Ini lebih sering terjadi pada *recurrent neural networks* (RNNs)[cite: 332].
    * [cite_start]**Penyebab**: Terutama disebabkan oleh kombinasi fungsi aktivasi *logistic sigmoid* (yang cenderung saturasi pada input besar, menghasilkan gradien mendekati nol) dan teknik inisialisasi bobot standar (distribusi normal dengan rata-rata 0 dan standar deviasi 1)[cite: 332]. [cite_start]Ini menyebabkan varians output lapisan jauh lebih besar dari varians inputnya, menyebabkan saturasi[cite: 332].

2.  **Glorot and He Initialization (Inisialisasi Glorot dan He)**:
    * [cite_start]**Solusi**: Untuk mengatasi masalah gradien yang tidak stabil, sinyal perlu mengalir dengan baik ke dua arah (*forward* dan *reverse*) tanpa menghilang atau meledak[cite: 333].
    * [cite_start]**Glorot Initialization (Xavier Initialization)**: Menginisialisasi bobot koneksi setiap lapisan secara acak dengan rata-rata 0 dan varians $1 / fan_{avg}$ atau distribusi seragam antara $\pm r$ dengan $r = \sqrt{3 / fan_{avg}}$, di mana $fan_{avg} = (fan_{in} + fan_{out}) / 2$[cite: 333]. [cite_start]Ini adalah kompromi yang baik dan sangat mempercepat pelatihan[cite: 334].
    * [cite_start]**He Initialization**: Mirip dengan Glorot, tetapi khusus untuk fungsi aktivasi ReLU dan variasinya, menggunakan varians $2 / fan_{in}$[cite: 334].
    * [cite_start]**LeCun Initialization**: Mirip dengan Glorot, tetapi menggunakan varians $1 / fan_{in}$, direkomendasikan untuk fungsi aktivasi SELU[cite: 334].
    * [cite_start]**Implementasi di Keras**: Dapat diatur melalui parameter `kernel_initializer` saat membuat lapisan `Dense`[cite: 334].

3.  **Nonsaturating Activation Functions (Fungsi Aktivasi Non-Saturasi)**:
    * [cite_start]**Masalah ReLU**: Meskipun cepat dihitung dan tidak saturasi untuk nilai positif, ReLU menderita masalah "dying ReLUs", di mana neuron berhenti mengeluarkan apa pun selain 0 karena bobotnya disetel sedemikian rupa sehingga jumlah tertimbang inputnya selalu negatif[cite: 335].
    * [cite_start]**Leaky ReLU**: Mengatasi masalah *dying ReLUs* dengan memiliki sedikit kemiringan ($\alpha$, biasanya 0.01 atau 0.2) untuk $z < 0$[cite: 335]. [cite_start]Variannya termasuk Randomized Leaky ReLU (RReLU) dan Parametric Leaky ReLU (PReLU)[cite: 335].
    * [cite_start]**ELU (Exponential Linear Unit)**: Mengungguli semua varian ReLU dalam eksperimen[cite: 336]. Mengambil nilai negatif untuk $z < 0$ (membantu masalah gradien menghilang), memiliki gradien non-nol untuk $z < 0$ (menghindari *dying neurons*), dan mulus di mana-mana jika $\alpha = 1$ (mempercepat Gradient Descent). [cite_start]Kerugiannya adalah lebih lambat dihitung[cite: 337].
    * [cite_start]**SELU (Scaled ELU)**: Versi skala dari ELU[cite: 337]. [cite_start]Jika DNN terdiri dari tumpukan lapisan *dense* dengan SELU dan inisialisasi LeCun normal serta input distandardisasi, jaringan akan *self-normalize* (output setiap lapisan cenderung mempertahankan rata-rata 0 dan standar deviasi 1), yang memecahkan masalah gradien yang menghilang/meledak[cite: 337].
    * **Panduan Praktis**: SELU > ELU > Leaky ReLU > ReLU > tanh > logistic. [cite_start]Namun, ReLU mungkin tetap menjadi pilihan terbaik jika kecepatan adalah prioritas utama karena optimasi hardware[cite: 338].

4.  **Batch Normalization (Normalisasi Batch)**:
    * [cite_start]**Teknik**: Menambahkan operasi di model, sebelum atau sesudah fungsi aktivasi setiap *hidden layer*, yang melakukan *zero-centering* dan normalisasi setiap input, lalu menskalakan dan menggeser hasilnya menggunakan dua vektor parameter baru (gamma $\gamma$ untuk scaling dan beta $\beta$ untuk shifting)[cite: 339].
    * **Manfaat**: Sangat mengurangi masalah gradien yang menghilang, membuat jaringan kurang sensitif terhadap inisialisasi bobot, memungkinkan *learning rate* yang lebih besar (mempercepat pembelajaran), dan bertindak sebagai regularizer (mengurangi kebutuhan akan teknik regularisasi lain).
    * [cite_start]**Cara Kerja saat Inferensi**: Menggunakan rata-rata bergerak (moving average) dari rata-rata input dan standar deviasi yang diestimasi selama pelatihan[cite: 340].
    * [cite_start]**Implementasi di Keras**: Cukup tambahkan lapisan `BatchNormalization`[cite: 341]. [cite_start]Ini menambah parameter non-trainable (rata-rata bergerak) dan trainable ($\gamma, \beta$)[cite: 342].
    * [cite_start]**Posisi BN Layer**: Ada perdebatan apakah lebih baik sebelum atau sesudah fungsi aktivasi[cite: 343]. [cite_start]BN layer dapat digabungkan dengan lapisan sebelumnya setelah pelatihan untuk menghindari penalti *runtime*[cite: 341].
    * **Hyperparameter**: `momentum` (mengontrol rata-rata bergerak) dan `axis` (menentukan sumbu normalisasi).

5.  **Gradient Clipping (Pemotongan Gradien)**:
    * [cite_start]**Teknik**: Memotong gradien selama *backpropagation* agar tidak melebihi *threshold* tertentu[cite: 345].
    * [cite_start]**Tujuan**: Mengatasi masalah *exploding gradients*[cite: 345]. [cite_start]Paling sering digunakan di RNNs di mana Batch Normalization sulit diterapkan[cite: 345].
    * [cite_start]**Implementasi di Keras**: Atur `clipvalue` atau `clipnorm` saat membuat *optimizer*[cite: 345]. [cite_start]`clipvalue` memotong setiap komponen gradien, `clipnorm` memotong seluruh gradien jika norma L2-nya melebihi *threshold*[cite: 345].

6.  **Reusing Pretrained Layers (Menggunakan Ulang Lapisan yang Sudah Dilatih)**:
    * **Transfer Learning**: Menggunakan lapisan bawah dari jaringan saraf yang sudah ada (dilatih untuk tugas serupa) dan menambahkan lapisan output baru untuk tugas yang sedang dikerjakan.
    * [cite_start]**Manfaat**: Mempercepat pelatihan secara signifikan dan membutuhkan lebih sedikit data pelatihan[cite: 346].
    * **Proses**: Ganti lapisan output model asli, bekukan lapisan yang digunakan kembali (buat bobotnya tidak dapat dilatih) untuk *epochs* awal, lalu *unfreeze* beberapa lapisan teratas dan lanjutkan pelatihan dengan *learning rate* yang lebih rendah untuk *fine-tuning*.
    * **Implementasi di Keras**: Muat model asli, buat model baru dengan lapisan-lapisan yang digunakan kembali dan lapisan output baru. Kloning model jika tidak ingin model asli ikut terpengaruh.

7.  **Unsupervised Pretraining (Pelatihan Awal Tanpa Pengawasan)**:
    * [cite_start]**Tujuan**: Untuk tugas kompleks dengan sedikit data berlabel, tetapi banyak data tidak berlabel[cite: 349].
    * [cite_start]**Teknik**: Latih model tanpa pengawasan (misalnya, *autoencoder* atau *generative adversarial network*) menggunakan semua data tidak berlabel, lalu gunakan kembali lapisan bawah model tanpa pengawasan tersebut, tambahkan lapisan output untuk tugas sebenarnya, dan *fine-tune* menggunakan pembelajaran terawasi dengan data berlabel[cite: 349].
    * [cite_start]Ini adalah teknik yang mengawali kebangkitan Deep Learning pada tahun 2006[cite: 349].

8.  **Pretraining on an Auxiliary Task (Pelatihan Awal pada Tugas Tambahan)**:
    * [cite_start]**Tujuan**: Ketika data berlabel untuk tugas utama sedikit, tetapi mudah mendapatkan data berlabel untuk tugas tambahan yang terkait[cite: 350].
    * [cite_start]**Teknik**: Latih jaringan saraf pertama pada tugas tambahan tersebut, kemudian gunakan kembali lapisan bawah jaringan itu untuk tugas utama dan *fine-tune*[cite: 350].
    * [cite_start]**Self-supervised learning**: Secara otomatis menghasilkan label dari data itu sendiri (contohnya adalah memprediksi kata yang hilang dalam kalimat), yang kemudian digunakan untuk melatih model menggunakan teknik pembelajaran terawasi[cite: 351]. [cite_start]Ini diklasifikasikan sebagai bentuk pembelajaran tanpa pengawasan[cite: 351].

9.  **Faster Optimizers (Pengoptimal yang Lebih Cepat)**:
    * [cite_start]**Tujuan**: Mempercepat pelatihan DNN yang sangat besar[cite: 351].
    * [cite_start]**Momentum Optimization**: Menambahkan "momentum" ke Gradient Descent, memperhitungkan gradien sebelumnya untuk mempercepat konvergensi di lembah yang dangkal dan menghindari *local optima*[cite: 351].
    * [cite_start]**Nesterov Accelerated Gradient (NAG)**: Varian *momentum optimization* yang mengukur gradien sedikit di depan arah momentum, seringkali lebih cepat[cite: 353].
    * **AdaGrad**: Menskalakan gradien ke bawah sepanjang dimensi yang paling curam, menghasilkan *adaptive learning rate*. [cite_start]Namun, seringkali berhenti terlalu cepat pada DNN[cite: 355].
    * [cite_start]**RMSProp**: Memperbaiki AdaGrad dengan hanya mengakumulasi gradien dari iterasi terbaru menggunakan peluruhan eksponensial[cite: 355]. [cite_start]Umumnya lebih baik dari AdaGrad[cite: 355].
    * [cite_start]**Adam (Adaptive Moment Estimation)**: Menggabungkan ide *momentum optimization* dan RMSProp[cite: 356]. [cite_start]Menjaga rata-rata bergerak gradien dan rata-rata bergerak kuadrat gradien[cite: 356]. [cite_start]Seringkali merupakan pilihan *default* yang bagus karena memerlukan sedikit *tuning* `learning_rate`[cite: 357].
    * [cite_start]**AdaMax**: Varian Adam yang menggantikan norma L2 dengan norma L-infinity (maksimum), dapat lebih stabil tergantung dataset[cite: 357].
    * [cite_start]**Nadam**: Adam + trik Nesterov, seringkali sedikit lebih cepat dari Adam[cite: 358].
    * [cite_start]**Catatan**: *Adaptive optimization methods* terkadang dapat menghasilkan solusi yang *generalize* dengan buruk pada beberapa dataset[cite: 358]. [cite_start]Coba NAG biasa jika ini terjadi[cite: 358].

10. **Learning Rate Scheduling (Penjadwalan Learning Rate)**:
    * **Tujuan**: Mencapai konvergensi yang lebih cepat dan solusi yang lebih baik daripada *learning rate* konstan.
    * **Power Scheduling**: $\eta(t) = \eta_0 / (1 + t/s)^c$. [cite_start]*Learning rate* turun dengan cepat, lalu melambat[cite: 360].
    * **Exponential Scheduling**: $\eta(t) = \eta_0 \cdot 0.1^{t/s}$. [cite_start]*Learning rate* turun secara eksponensial[cite: 360].
    * [cite_start]**Piecewise Constant Scheduling**: Menggunakan *learning rate* konstan untuk sejumlah *epochs*, lalu *learning rate* yang lebih kecil untuk *epochs* berikutnya[cite: 361].
    * [cite_start]**Performance Scheduling**: Mengurangi *learning rate* ketika *validation error* berhenti turun[cite: 361].
    * [cite_start]**1cycle Scheduling**: Meningkatkan *learning rate* dari rendah ke tinggi di paruh pertama pelatihan, lalu menurunkannya lagi di paruh kedua, sambil membalik momentum[cite: 361]. [cite_start]Seringkali sangat mempercepat pelatihan dan menghasilkan kinerja lebih baik[cite: 361].
    * **Implementasi di Keras**: Menggunakan parameter `decay` pada *optimizer* (untuk *power scheduling*), atau `LearningRateScheduler` callback dengan fungsi jadwal kustom, atau `keras.optimizers.schedules` (spesifik `tf.keras`).

11. **Avoiding Overfitting Through Regularization (Menghindari Overfitting Melalui Regularisasi)**:
    * [cite_start]**Tujuan**: Mengurangi *overfitting* pada model DNN yang memiliki banyak parameter[cite: 364].
    * [cite_start]**Early Stopping**: Menghentikan pelatihan segera setelah *validation error* mencapai minimum[cite: 364].
    * [cite_start]**Batch Normalization**: Selain mengatasi masalah gradien, juga berfungsi sebagai regularizer[cite: 341].
    * [cite_start]**$l_1$ and $l_2$ Regularization**: Menambahkan istilah penalti ke fungsi *loss* untuk menjaga bobot model tetap kecil[cite: 364]. [cite_start]$l_1$ menghasilkan model yang *sparse* (banyak bobot nol), $l_2$ (Ridge) mendorong bobot kecil[cite: 364].
        * [cite_start]**Implementasi di Keras**: `kernel_regularizer=keras.regularizers.l1(0.01)` atau `keras.regularizers.l2(0.01)`[cite: 364].
    * [cite_start]**Dropout**: Pada setiap langkah pelatihan, setiap neuron (kecuali output) memiliki probabilitas $p$ untuk "dihilangkan" sementara[cite: 365]. [cite_start]Ini memaksa neuron untuk tidak bergantung pada neuron tetangga, menciptakan jaringan yang lebih *robust* dan *generalize* lebih baik[cite: 366].
        * [cite_start]**Dropout Rate ($p$)**: Biasanya antara 10% dan 50%[cite: 365].
        * [cite_start]**Implementasi di Keras**: `keras.layers.Dropout(rate=p)`[cite: 367].
    * [cite_start]**Monte Carlo (MC) Dropout**: Meningkatkan kinerja model *dropout* yang sudah dilatih tanpa melatih ulang, dan memberikan ukuran ketidakpastian model yang lebih baik[cite: 368]. [cite_start]Dilakukan dengan membuat banyak prediksi pada waktu inferensi dengan *dropout* diaktifkan (`training=True`) dan merata-ratakan hasilnya[cite: 368].
    * [cite_start]**Max-Norm Regularization**: Untuk setiap neuron, membatasi norma L2 dari bobot koneksi masukannya ($\lVert \mathbf{w} \rVert_2 \le r$, di mana $r$ adalah hyperparameter *max-norm*)[cite: 370]. [cite_start]Tidak menambah istilah *loss*, tetapi menskalakan bobot jika perlu setelah setiap langkah pelatihan[cite: 370]. [cite_start]Membantu mengurangi *overfitting* dan menstabilkan gradien[cite: 371].
        * [cite_start]**Implementasi di Keras**: `kernel_constraint=keras.constraints.max_norm(r)`[cite: 371].

12. **Summary and Practical Guidelines (Ringkasan dan Panduan Praktis)**:
    * [cite_start]**Default DNN Configuration**: Tabel 11-3 menyajikan konfigurasi DNN default yang bekerja baik di sebagian besar kasus: He initialization, ELU activation, Batch Norm (jika dalam), Early stopping (+L2 reg.), Momentum optimization (atau RMSProp/Nadam), 1cycle scheduling[cite: 371].
    * [cite_start]**Self-Normalizing Net Configuration**: Tabel 11-4 untuk jaringan *self-normalizing*: LeCun initialization, SELU activation, tanpa normalisasi eksplisit (karena *self-normalization*), Alpha Dropout (jika perlu), Momentum optimization (atau RMSProp/Nadam), 1cycle scheduling[cite: 372].
    * [cite_start]**Penting**: Normalisasi fitur input, penggunaan ulang model *pretrained*, *unsupervised pretraining*, atau *pretraining* pada tugas tambahan[cite: 372].
    * [cite_start]**Kasus Khusus**: Untuk model *sparse* ($l_1$ regularization, TF Model Optimization Toolkit), model *low-latency* (lebih sedikit lapisan, gabungkan BN, fungsi aktivasi cepat, kuantisasi), atau aplikasi *risk-sensitive* (MC Dropout)[cite: 372].

## Analisis dan Relevansi untuk Mahasiswa

Bab 11 adalah panduan esensial untuk melatih Jaringan Saraf Tiruan yang dalam secara efektif. Dengan cakupan yang luas mulai dari inisialisasi bobot dan fungsi aktivasi hingga berbagai teknik optimasi dan regularisasi, bab ini membekali mahasiswa dengan pengetahuan dan alat yang diperlukan untuk membangun dan menyetel model DNN berkinerja tinggi, serta mengatasi masalah-masalah umum yang mungkin terjadi selama pelatihan. Pemahaman mendalam dari bab ini adalah kunci untuk menjadi praktisi Deep Learning yang kompeten.

* **Mengatasi Tantangan Nyata**: Mahasiswa akan dihadapkan pada masalah *vanishing/exploding gradients* dan *overfitting* dalam proyek DNN nyata. [cite_start]Bab ini memberikan *toolkit* komprehensif untuk mendiagnosis dan mengatasi masalah-masalah ini[cite: 331].
* [cite_start]**Fondasi Arsitektur Modern**: Teknik-teknik seperti Batch Normalization dan fungsi aktivasi non-saturasi adalah blok bangunan penting dalam arsitektur DNN modern[cite: 339, 335]. Memahaminya bukan hanya untuk perbaikan model, tetapi juga untuk memahami mengapa model-model saat ini didesain seperti itu.
* **Optimalisasi Pelatihan**: Berbagai *optimizer* canggih (Momentum, Adam, Nadam, dll.) dan strategi penjadwalan *learning rate* (1cycle scheduling) memberikan mahasiswa alat untuk mempercepat pelatihan dan mencapai konvergensi yang lebih baik. Ini sangat krusial untuk efisiensi komputasi.
* **Regularisasi Komprehensif**: Selain *early stopping*, bab ini memperkenalkan *L1/L2 regularization*, *dropout*, *MC Dropout*, dan *max-norm regularization*. Mahasiswa belajar tentang berbagai cara untuk memerangi *overfitting*, yang merupakan salah satu tantangan terbesar dalam DNN.
* **Transfer Learning dan Pretraining**: Konsep *transfer learning*, *unsupervised pretraining*, dan *pretraining* pada tugas tambahan adalah teknik tingkat lanjut yang memungkinkan mahasiswa melatih model yang efektif bahkan dengan data berlabel terbatas. Ini adalah keahlian yang sangat dicari di industri.
* **Panduan Praktis**: Adanya bagian ringkasan dan panduan praktis sangat membantu mahasiswa dalam menyaring informasi yang banyak dan menerapkan rekomendasi terbaik secara langsung ke proyek mereka.

## Kesimpulan

Bab 11 adalah panduan esensial untuk melatih Jaringan Saraf Tiruan yang dalam secara efektif. Dengan cakupan yang luas mulai dari inisialisasi bobot dan fungsi aktivasi hingga berbagai teknik optimasi dan regularisasi, bab ini membekali mahasiswa dengan pengetahuan dan alat yang diperlukan untuk membangun dan menyetel model DNN berkinerja tinggi, serta mengatasi masalah-masalah umum yang mungkin terjadi selama pelatihan. Pemahaman mendalam dari bab ini adalah kunci untuk menjadi praktisi Deep Learning yang kompeten.


# REPRODUCE CODE

In [1]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
from functools import partial # Untuk RegularizedDense

# Mengatur seed untuk reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# --- BAGIAN 0: Persiapan Data (dari Chapter 10) ---
print("--- Persiapan Data Fashion MNIST ---")
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0 # Skala test set juga

# Skalakan X_train untuk SELU (mean 0, std 1)
# Ini penting untuk self-normalizing networks (SELU)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 28 * 28)).reshape(-1, 28, 28)
X_valid_scaled = scaler.transform(X_valid.reshape(-1, 28 * 28)).reshape(-1, 28, 28)
X_test_scaled = scaler.transform(X_test.reshape(-1, 28 * 28)).reshape(-1, 28, 28)

print("Data Fashion MNIST siap.")


# --- BAGIAN 1: Glorot dan He Initialization ---
print("\n--- Bagian 1: Glorot dan He Initialization ---")

# Contoh penggunaan He initialization dengan relu
# Default Keras adalah Glorot uniform
model_he_init = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"), # He Normal
    keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model_he_init.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
print("Model dengan He Initialization dibuat.")
model_he_init.summary()

# Contoh penggunaan VarianceScaling untuk kustomisasi lebih lanjut (misal He uniform fan_avg)
# Ini adalah contoh, tidak digunakan dalam model utama di bawah
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
print("\nInisialisasi kustom VarianceScaling (He uniform fan_avg) dibuat.")


# --- BAGIAN 2: Nonsaturating Activation Functions (ELU, Leaky ReLU, SELU) ---
print("\n--- Bagian 2: Nonsaturating Activation Functions ---")

# Model dengan ELU activation
model_elu = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model_elu.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
print("Model dengan ELU activation dibuat.")
model_elu.summary()

# Model dengan Leaky ReLU activation
model_leaky_relu = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2), # Tambahkan LeakyReLU layer
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model_leaky_relu.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
print("Model dengan Leaky ReLU activation dibuat.")
model_leaky_relu.summary()

# Model dengan SELU activation (membutuhkan LeCun Normal Initialization)
# Pastikan X_train_scaled dan X_valid_scaled digunakan
model_selu = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model_selu.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
print("Model dengan SELU activation (LeCun Normal) dibuat.")
model_selu.summary()

# Latih model ELU sebagai contoh (pelatihan bisa lama)
# print("\nMelatih model ELU (demo singkat)...")
# history_elu = model_elu.fit(X_train, y_train, epochs=5,
#                             validation_data=(X_valid, y_valid))
# print("Pelatihan model ELU selesai.")


# --- BAGIAN 3: Batch Normalization ---
print("\n--- Bagian 3: Batch Normalization ---")

# Model dengan Batch Normalization (BN) setelah setiap layer Dense, dan sebagai layer pertama
model_bn_after = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(), # BN sebagai layer pertama
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])
model_bn_after.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
print("Model dengan Batch Normalization (setelah aktivasi/dense) dibuat.")
model_bn_after.summary()

# Model dengan Batch Normalization sebelum aktivasi
# Dense layer tanpa bias (use_bias=False)
model_bn_before = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False), # Tanpa bias
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"), # Aktivasi setelah BN
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])
model_bn_before.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
print("Model dengan Batch Normalization (sebelum aktivasi) dibuat.")
model_bn_before.summary()

# Periksa parameter trainable dari layer BN pertama (hanya gamma dan beta)
# gamma dan beta adalah trainable, moving_mean dan moving_variance tidak
first_bn_layer = model_bn_after.layers[1]
print(f"\nParameter layer BN pertama: {[(var.name, var.trainable) for var in first_bn_layer.variables]}")


# --- BAGIAN 4: Gradient Clipping ---
print("\n--- Bagian 4: Gradient Clipping ---")

# Optimizer dengan clipvalue
optimizer_clip_value = keras.optimizers.SGD(clipvalue=1.0)
model_clip_value = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_clip_value.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_clip_value)
print("Model dengan optimizer clipvalue=1.0 dibuat.")

# Optimizer dengan clipnorm
optimizer_clip_norm = keras.optimizers.SGD(clipnorm=1.0)
model_clip_norm = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_clip_norm.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_clip_norm)
print("Model dengan optimizer clipnorm=1.0 dibuat.")


# --- BAGIAN 5: Reusing Pretrained Layers (Transfer Learning) ---
print("\n--- Bagian 5: Reusing Pretrained Layers (Transfer Learning) ---")

# Ini adalah contoh konseptual dari buku.
# Model A yang sudah dilatih (disimulasikan di sini)
# Model A dilatih pada 8 kelas Fashion MNIST (misal tanpa sandal dan shirt)
# Kita akan membuat dummy model_A
model_A = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(8, activation="softmax") # 8 kelas
])
model_A.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
# Latih model_A secara singkat untuk bobot
# model_A.fit(X_train[:1000], y_train[:1000] % 8, epochs=1) # Data dan label dummy
# model_A.save("my_model_A.h5") # Simpan model_A

# Memuat model A dan membuat model B_on_A
# model_A_loaded = keras.models.load_model("my_model_A.h5") # Asumsi my_model_A.h5 ada
# model_B_on_A = keras.models.Sequential(model_A_loaded.layers[:-1]) # Reused layers
# model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid")) # Output layer baru untuk task B (binary classification)

# Atau, kloning model A (jika tidak ingin berbagi layer)
# model_A_clone = keras.models.clone_model(model_A_loaded)
# model_A_clone.set_weights(model_A_loaded.get_weights())
# model_B_on_A_cloned = keras.models.Sequential(model_A_clone.layers[:-1])
# model_B_on_A_cloned.add(keras.layers.Dense(1, activation="sigmoid"))

print("Konsep transfer learning dijelaskan.")
print("Kode lengkap membutuhkan dataset dan model A yang spesifik.")


# --- BAGIAN 6: Faster Optimizers (Momentum, NAG, Adam, RMSprop) ---
print("\n--- Bagian 6: Faster Optimizers ---")

# Momentum Optimization
optimizer_momentum = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
model_momentum = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_momentum.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_momentum)
print("Model dengan Momentum Optimizer dibuat.")

# Nesterov Accelerated Gradient (NAG)
optimizer_nag = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model_nag = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_nag.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_nag)
print("Model dengan NAG Optimizer dibuat.")

# RMSprop
optimizer_rmsprop = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
model_rmsprop = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_rmsprop.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_rmsprop)
print("Model dengan RMSprop Optimizer dibuat.")

# Adam
optimizer_adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model_adam = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_adam.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_adam)
print("Model dengan Adam Optimizer dibuat.")

# Latih model Adam sebagai contoh (pelatihan bisa lama)
# print("\nMelatih model Adam (demo singkat)...")
# history_adam = model_adam.fit(X_train, y_train, epochs=5,
#                               validation_data=(X_valid, y_valid))
# print("Pelatihan model Adam selesai.")


# --- BAGIAN 7: Learning Rate Scheduling ---
print("\n--- Bagian 7: Learning Rate Scheduling ---")

# Power Scheduling (melalui decay di optimizer)
optimizer_power_decay = keras.optimizers.SGD(learning_rate=0.01, decay=1e-4) # decay = 1/s
model_power_decay = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_power_decay.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_power_decay)
print("Model dengan Power Scheduling (decay) dibuat.")

# Exponential Scheduling (melalui LearningRateScheduler callback)
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20) # learning_rate = 0.01, turun 10x setiap 20 epoch

lr_scheduler_exp = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
model_exp_decay = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_exp_decay.compile(loss="sparse_categorical_crossentropy", optimizer="sgd") # Optimizer dengan LR awal
print("Model dengan Exponential Scheduling dibuat.")
# history_exp_decay = model_exp_decay.fit(X_train, y_train, epochs=5,
#                                         validation_data=(X_valid, y_valid),
#                                         callbacks=[lr_scheduler_exp])


# Piecewise Constant Scheduling (melalui LearningRateScheduler callback)
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

lr_scheduler_piecewise = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)
model_piecewise = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_piecewise.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
print("Model dengan Piecewise Constant Scheduling dibuat.")
# history_piecewise = model_piecewise.fit(X_train, y_train, epochs=20,
#                                         validation_data=(X_valid, y_valid),
#                                         callbacks=[lr_scheduler_piecewise])

# ReduceLROnPlateau callback (Performance Scheduling)
lr_scheduler_plateau = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
model_plateau = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_plateau.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
print("Model dengan ReduceLROnPlateau (Performance Scheduling) dibuat.")
# history_plateau = model_plateau.fit(X_train, y_train, epochs=20,
#                                     validation_data=(X_valid, y_valid),
#                                     callbacks=[lr_scheduler_plateau])

# ExponentialDecay dari keras.optimizers.schedules (TF Keras-specific)
s = 20 * len(X_train) // 32 # Langkah dalam 20 epoch, batch size 32
learning_rate_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=s,
    decay_rate=0.1
)
optimizer_schedule = keras.optimizers.SGD(learning_rate=learning_rate_schedule)
model_schedule = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_schedule.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_schedule)
print("Model dengan ExponentialDecay (schedules API) dibuat.")


# --- BAGIAN 8: Avoiding Overfitting Through Regularization ---
print("\n--- Bagian 8: Regularization ---")

# L1 dan L2 Regularization
# Menggunakan functools.partial untuk mengurangi pengulangan kode
RegularizedDense = partial(keras.layers.Dense,
                         activation="elu",
                         kernel_initializer="he_normal",
                         kernel_regularizer=keras.regularizers.l2(0.01)) # L2 regularization

model_l2_reg = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax",
                     kernel_initializer="glorot_uniform") # Output layer biasanya glorot
])
model_l2_reg.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
print("Model dengan L2 Regularization dibuat.")
model_l2_reg.summary()


# Dropout Regularization
model_dropout = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2), # Dropout setelah Flatten
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2), # Dropout setelah hidden layer 1
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2), # Dropout setelah hidden layer 2
    keras.layers.Dense(10, activation="softmax")
])
model_dropout.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
print("Model dengan Dropout dibuat.")
model_dropout.summary()

# Monte Carlo (MC) Dropout (konseptual)
# Untuk model_dropout yang sudah dilatih
# print("\nMonte Carlo Dropout (konseptual):")
# # Ini harusnya dilakukan setelah model_dropout dilatih
# # Misalnya, setelah: history_dropout = model_dropout.fit(X_train, y_train, epochs=10)
# # Jika Anda ingin menjalankan ini, latih model_dropout terlebih dahulu

# # Membuat 100 prediksi dengan dropout aktif
# if 'model_dropout' in locals() and hasattr(model_dropout, 'history'): # Check if model_dropout was trained
#     y_probas_mc = np.stack([model_dropout(X_test_scaled, training=True) for sample in range(100)])
#     y_proba_mc = y_probas_mc.mean(axis=0)
#     y_std_mc = y_probas_mc.std(axis=0)
#     print(f"Probabilitas prediksi MC Dropout untuk X_test_scaled[0] (rata-rata): {y_proba_mc[0].round(2)}")
#     print(f"Standar deviasi MC Dropout untuk X_test_scaled[0]: {y_std_mc[0].round(2)}")
# else:
#     print("Lewati MC Dropout: model_dropout belum dilatih.")

# Custom MCDropout layer (jika model berisi BatchNormalization atau layer training=True lainnya)
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)
print("\nCustom MCDropout layer class dibuat.")

# Max-Norm Regularization
model_max_norm = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal",
                       kernel_constraint=keras.constraints.max_norm(1.)), # Max-norm constraint
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                       kernel_constraint=keras.constraints.max_norm(1.)),
    keras.layers.Dense(10, activation="softmax")
])
model_max_norm.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
print("Model dengan Max-Norm Regularization dibuat.")
model_max_norm.summary()

print("\n--- Semua contoh kode dari Chapter 11 telah direproduksi. ---")
print("Catatan: Beberapa bagian memerlukan pelatihan penuh atau file yang disimpan.")

--- Persiapan Data Fashion MNIST ---
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Data Fashion MNIST siap.

--- Bagian 1: Glorot dan He Initialization ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model dengan He Initialization dibuat.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_20 (Flatten)            │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 266,610 (1.02 MB)

 Trainable params: 266,610 (1.02 MB)

 Non-trainable params: 0 (0.00 B)


--- Semua contoh kode dari Chapter 11 telah direproduksi. ---
Catatan: Beberapa bagian memerlukan pelatihan penuh atau file yang disimpan.
